In [1]:
# sm on 3 April 2018, for childes-db draft

In [2]:
import pandas as pd
import MySQLdb
import sqlalchemy
import mysql_config
import numpy as np
import sys
import lexical_diversity
from joblib import Parallel, delayed
import multiprocessing

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.2.2 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [3]:
def extract(utterance):
    return(utterance.split(' '))

def flatten(gloss_arrays):
    return([item for sublist in gloss_arrays for item in sublist])

def num_types(gloss_array):
    return(len(set(gloss_array)))

In [4]:
auth = mysql_config.Authenticator('ec2')

In [5]:
con= MySQLdb.connect(host=auth.host, 
                port=auth.port,user=auth.user, passwd=auth.passwd, 
                db='childesdb')

In [6]:
# load utterances into memory, about 3 minutes on Chompsky; 16.5m * 3 array; don't do it on little machines
utterances = pd.read_sql("select collection_id, transcript_id, corpus_id, speaker_role, gloss from utterance", con)

In [7]:
utterances.iloc[0:5]

,collection_id,transcript_id,corpus_id,speaker_role,gloss
0,1,1,1,Adult,muy bien Diana
1,1,2,1,Adult,hab�a una vez una ni�a
2,1,1,1,Adult,qui�n gan�
3,1,1,1,Child,la roja
4,1,2,1,Adult,que le ten�a mucho miedo a la oscuridad


In [8]:
utterances['gloss_array'] = [extract(x) for x in utterances.gloss]

In [9]:
transcripts = utterances.groupby(['transcript_id','speaker_role']).gloss_array.agg(flatten).reset_index()
transcripts['num_tokens'] = [len(x) for x in transcripts.gloss_array]    

In [10]:
transcripts.iloc[0:5]


,transcript_id,speaker_role,gloss_array,num_tokens
0,1,Adult,"[muy, bien, Diana, qui�n, gan�, la, roja, porq...",200
1,1,Child,"[la, roja, porque, ella, la, explic�, mejor, m...",160
2,2,Adult,"[hab�a, una, vez, una, ni�a, que, le, ten�a, m...",17
3,2,Child,"[en, eso, se, fue, la, luz, entonces, se, espa...",233
4,3,Adult,"[qui�n, gan�, el, amarillo, porqu�, porqu�, fu...",160


In [11]:
def compute_lexical_measures(row):
    return({'hdd': lexical_diversity.hdd(row['gloss_array']),
           'mtld':lexical_diversity.mtld(row['gloss_array']),
           'num_types': num_types(row['gloss_array']),
           'transcript_id': row['transcript_id'],
            'speaker_role': row['speaker_role'],
           'num_tokens': row['num_tokens'],
           })

In [ ]:
# serial version -- lower memory usage

#transcripts['mtld'] = [lexical_diversity.mtld(x) for x in transcripts.gloss_array]
#transcripts['hdd'] = [lexical_diversity.hdd(x) for x in transcripts.gloss_array]
#transcripts['num_types'] = [num_types(x) for x in transcripts.gloss_array]
#transcripts['ttr'] = transcripts['num_types'] / transcripts['num_tokens']

In [12]:
# parallel version -- not much improvement because lots of jobs / overhead
lex_measures = pd.DataFrame(Parallel(n_jobs=multiprocessing.cpu_count())(delayed(compute_lexical_measures)(i) for i in transcripts.to_dict('records')))

lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, sample)
lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, sample)
lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, sample)
lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, sample)
lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, sample)
lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, sample)
lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, sample)
lexical_diversity.py:63: RuntimeWarning: invalid value encountered in double_scalars
  scipy.special.comb(population, 

In [13]:
lex_measures['ttr'] = lex_measures.num_types / lex_measures.num_tokens

In [15]:
lex_measures.iloc[0:5]

,hdd,mtld,num_tokens,num_types,speaker_role,transcript_id,ttr
0,0.736008,10.526316,200,82,Adult,1,0.410000
1,0.763520,11.428571,160,79,Child,1,0.493750
2,NaN,NaN,17,16,Adult,2,0.941176
3,0.812148,12.944444,233,114,Child,2,0.489270
4,0.754292,11.428571,160,75,Adult,3,0.468750


In [ ]:
#[ ] write back to the database
#lex_measures.to_sql(name='!!!', con=con, if_exists='replace')
# this would replace the stuff in transcript_by_speaker...